In [ ]:
# bring in a few jupyter packages that are helpful for visualizations and reloading packages
%load_ext tikzmagic
%load_ext autoreload

# Modeling for Experimental Outburst Floods

## Overview

Inspired by landforms present in the Upper Peninsula of Michigan (shown below), we are interested in understanding how sediment and water move through a confined spillway with an erodible bed, and thus produce bedforms at the downstream outlet. To build an appropriate model for parameterizing the system, let us consider a lake that starts draining catastrophically through a non-erodible spillway into another standing body of water. 

## Modeling Strategy

For design, parameterization, and evaluation purposes, we will consider a variety of cases:
1. The levels of each lake are held constant, and the discharge is constant through the system. 
2. The level of each lake is allowed to vary as the volume of that basin is filled. 
3. The level of the upper lake falls as the lake drains, and the downstream lake is either held constant either at (a) a fixed positive value or (b) at zero. 

For all of these, we will not allow the bed to erode in our model. This is simply a practical consideration. While there exist a few models for computing erosion in a situation like this, adapting those models to this model setup and implementing them will be a substantial task with no clear benefit. Rather, our purpose in constructing this model is to constrain a few parameters like roughness from trial tests. Then, when we use sediment in the spillways, we can use this model framework to estimate (very approximately) the discharge and Froude number at the spillway outlet in a given moment through the experiment.

So, we need a flexible model framework that can meet each of the above scenarios and compute discharge and water levels, and then evolve to a final state. 

## Experimental Setup and Model Domain

We constrain the geometry of the model system to match our flume setup. Namely, we have inserted a narrow spillway into an 11 m-long 6 in-wide flume, and installed a lift-gate at the upstream end of the spillway. This creates two pools, one upstream and one downstream of the spillway. By changing the position of the spillway in the flume, we can change the relative sizes of the two pools. See sketches below of the setup.

In [ ]:
%%tikz -s 1000,500 -sc 1.5

\fill[blue!30!white] (0,0) rectangle (4,1);
\fill[blue!30!white] (6,0) rectangle (10, 1);
\fill[black!30!white] (4,1) rectangle (6, 0.66);
\fill[black!30!white] (4,0) rectangle (6, 0.33);

\draw[thick] (4,1) -- (4,0.66) -- (6, 0.66) -- (6, 1);
\draw[thick] (4,0) -- (4,0.33) -- (6, 0.33) -- (6, 0);
\draw[thick, ->] (4,1.5) node[anchor=south] {\footnotesize lift-gate} -- (4,1.1);
\draw[thick] (0,0) rectangle (10,1);

\node at (5,0.5) {\footnotesize spillway} ;
\node at (2,0.5) {\footnotesize upstream pool} ;
\node at (8,0.5) {\footnotesize downstream pool} ;

In [ ]:
%%tikz -s 1000,500 -sc 1.5

\fill[blue!30!white] (0,0) -- (0,0.7) -- (4, 0.7) -- (4, 0.5) -- (3, 0) -- cycle;
\fill[blue!30!white] (7,0) -- (5.2,0.3) -- (10, 0.3) -- (10, 0) -- cycle;
\fill[black!30!white] (3, 0) -- (4, 0.5) -- (7, 0)  -- cycle;
\draw (3, 0) -- (4, 0.5) -- (7, 0);
\draw[thick] (0,1) -- (0,0) -- (10, 0) -- (10, 1);
\draw[thick, ->] (4,0.5) -- (4,1) node[anchor=south] {\footnotesize lift-gate};

## Model Framework

We consider flow to rapidly establish a steady-state condition within the spillway (no shock waves). Flow through the spillway structure between the two pools is a function of (a) the hydraulic head between the two pools and (b) the geometry of the spillway. Using classic open-channel hydraulics calculations, we can compute the discharge and flow profile between the two pools. To demonstrate how the computations work, let's evaluate the situation in scenario **1.** above, where the level of each pool is constant, and we need to calculate the discharge.

Given the following parameters for the flume setup:

| Thing | value   |
|------|------|
| Flume Depth | 15 inches (0.381 m) |
| Flume Width | 6 inches (0.1524 m) |
| Flume Length | about 11 m |
| Basement Slope | 0 |
| Spillway height| 8" (0.2032 m) |
| Spillway length| 8' (2.44 m) |
| Spillway width | 3.24 cm |
| Roughness coefficient (n) | 0.015 | 
| Upstream pool length | 4.5 m |
| Downstream pool length | 4.06 m |

I have built this entire process into a python class called `Spillway`. Let's consider the simplest case, where water is impounded upstream of the spillway and there is no water in the downstream spillway. The initial condition is plotted with vertical exaggeration.

In [ ]:
%autoreload
import spillway as sp

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

In [ ]:
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(8*2.54*1e-2) # set up the spillway with elevation fall of 8 inches
spillway.set_y_values(0.15, 0.0) # set water levels
fig = spillway.show_current_condition() # method that displays the system state as a cartoon

In this condition, if the gate is lifted, water will begin to spill from the upstream pool to the downstream pool. An important question to resolve at this stage is "how fast?"

### The case of supercritical flow

To answer this question, we should consider the energy balance of the water as it moves down the spillway. The total energy of the flow (expressed as head, $H$) is

$$ H = z + y + \frac{v^2}{2g}, $$ 

where $z$ is the potential head (elevation of the bed of the channel), $y$ is the pressure head from the weight of the water in the channel, and $\frac{v^2}{2g}$ is the velocity head. This is a statement of the total energy the water has. If we normalize the energy to the elevation, and thus evaluate changes in energy along the channel, we obtain the "specific energy" $E$, given as 

$$ E = y + \frac{v^2}{2g}. $$

By imposing flow continuity ($Q = A \times v$), we can restate the velocity in terms of the cross-sectional area and discharge. Moreover, since the spillway is known to be rectangular with width $b$, we can write

$$ E = y + \frac{Q^2}{2gA^2} = y + \frac{Q^2}{2gb^2y^2}. $$

From here, we can see that if the discharge is given, we can construct a curve that relates $E$ and $y$. Let's plot that. We can clearly see that above a threshold, for any given flow energy, there are two possible alternate depths. 

In [ ]:
def spec_E(y, Q, b, g = 9.8):
    E = y + (Q * Q) / (2 * g * b * b * y * y)
    return E

q = 0.001    # 1 L/s 
b = spillway.b   # width of spillway
y = np.arange(0.015, 0.2, 0.001) # plot a range of depth values

def plot_specific_energy_curve(y, q, b): 
    E = spec_E(y, q, spillway.b) # compute E

    fig, ax = plt.subplots(figsize = (5, 5))
    ax.plot(E, y, 'k-', linewidth = 2)
    ax.plot(y, y, '-', color = '0.5')
    ax.set_xlabel('specific energy (m)')
    ax.set_ylabel('flow depth (m)')

    ax.annotate(
        "y = E", 
        (max(y) * 0.35, max(y) * 0.4), 
        rotation = 45, color = '0.5', ha = 'center', va = 'center'
    )

    ax.annotate(
        "Q = " + str(round(q*1000, 4)) + " L/s", xy = (0.9, 0.05), 
        xytext = (0.95, 0.05), textcoords = 'axes fraction', 
        xycoords = 'axes fraction', 
        color = 'k', ha = 'right', va = 'center'
    )
    
    ax.axis('equal')
    
    return fig, ax

fig, ax = plot_specific_energy_curve(y, q, b)

E_example = 0.13
objective = lambda y, qq, bb: spec_E(y, qq, bb) - E_example
y_subcrit = root_scalar(objective, args = (q, b), bracket = [0.1, 0.15]).root
y_supcrit = root_scalar(objective, args = (q, b), bracket = [0.01, 0.03]).root
plt.plot([E_example, E_example], [y_subcrit, y_supcrit], 'ro--')

ann = ax.annotate(
    "Alternate\nflow depths", 
    (E_example * 1.05, np.mean([y_subcrit, y_supcrit])), 
    rotation = 0, color = 'r', ha = 'left', va = 'center'
)


The deeper and shallower alternate depths for the same flow state correspond, respectively, to subcritical flow and supercritical flow. There is one flow depth where the specific energy is at a minimum, where the flow is said to be critical. We can compute this directly by finding the minimum of $E$ with respect to $y$. Given that

$$ E = y + \frac{Q^2}{2gb^2y^2}, $$

$$ \frac{dE}{dy} = 1 - 2\frac{Q^2}{2gb^2y^3} = 0, $$

$$ 1 = \frac{Q^2}{gb^2y^3}, $$ 

$$ y^3 = \frac{Q^2}{gb^2}, $$ 

which now gives a formula for the critical depth ($y_c$), as

$$ y_c = \frac{Q^2}{gb^2}^{\frac{1}{3}}. $$ 

returning to the formula for specific energy, we can see that

$$ E = y + \frac{1}{2y^2}\frac{Q^2}{gb^2} = y + \frac{1}{2y^2}y^3 = \frac{3}{2}y_c$$

where the critical depth ($y_c$) is now clearly a constant ratio of the specific energy. We can plot this as a line on our figure, and indicate the depth of critical flow as the intersection of those two lines.

In [ ]:
def critical_depth(q, b, g = 9.8):
    y = ((q*q) / (g * b*b))**(1/3)
    return y

yc = critical_depth(q, b)

fig, ax = plot_specific_energy_curve(y, q, b)

ax.plot(y, 2/3*y, '0.5')
ax.plot([min(y), spec_E(yc, q, b)], [yc, yc], 'o--', color = 'xkcd:red')

y_ann = max(y) * 0.75

_ = ax.annotate(
    "y = 2/3 E", 
    (y_ann * 1.1, y_ann*2/3), 
    rotation = 33, color = '0.5', ha = 'center', va = 'center'
)

_ = ax.annotate(
    "Critical flow depth", 
    (spec_E(yc, q, b) * 1.1, yc), 
    rotation = 0, color = 'xkcd:red', ha = 'left', va = 'center'
)

Furthermore, we can take our equation for the critical depth, 

$$ y^3 = \frac{Q^2}{gb^2}, $$ 

and by rearranging and substituting $A = by$ and $v = \frac{Q}{A}$, 

$$ y = \frac{Q^2}{g(by)^2} = \frac{v^2}{g} $$ 

we can recognize that

$$ 1 = \frac{v^2}{gy} = Fr $$

Thus, for open-channel flow, the critical depth gives the Froude number as unity. If $Fr \ne 1$, then flow can be either subcritical or supercritical, which will have very different implications for computing the discharge when we consider the impact of the downstream pool on the flow. 

Importantly, we do not know _a priori_ whether the flow will be supercritical or not based on the initial conditions imposed on `spillway` above. Subcritical flow conditions are established when the slope is shallow enough, and the resistance to flow is high enough such that the flow with low potential energy is preferred (higher alternate depth). To account for resistance and slope, we need a formula that relates the velocity of the flow to the slope and depth. The Manning equation is the preferred formula.

To use this, we make the simplifying assumption that flow _is uniform_, that is, $y_a = y_b = y_n$ for a given slope $S$ at every cross section $a$ and $b$. This is also called "normal" flow. The Manning formula can then give us the discharge as

$$ Q = \frac{AR^{2/3}}{n} \sqrt{S} $$

where $A$ is the cross-sectional area, $S$ is the slope, $R$ is the hydraulic radius, and $n$ is Manning's $n$. Thus, substituting in the formulae for a rectangular channel,

$$ Q = \frac{by_n\left(\frac{by_n}{2y_n + b}\right)^{2/3}}{n} \sqrt{S} $$

For our problem, the roughness, width, and slope are all given, so this equation allows us to solve for $Q_n$ (the discharge for a given flow depth provided that the flow is uniform). We can also numerically solve this equation to get a normal flow depth for a given discharge. The uniform flow depth will be one of either the supercritical or subcritical alternate depths. 

`find_normal_flow_depth()` is a method of the `Spillway` class, that returns the uniform flow depth when given a discharge value. It finds this using a numerical root-finding algorithm to solve the equation above. (slope, width, and roughness are attributes of the class.)

Now, for the first scenario where we open the spillway to initiate flow, we do not know the discharge a priori, but what we do know is the total specific energy at the upstream end of the spillway, $E$, because this is just the height of the water over the crest of the spillway. We can call this section **a**, and the downstream exit of the spillway section **b**, and thus the specific energy at section **a** will be $y_a$. So, we can use the equations above to show that if the water is simply allowed to fall down the spillway, the critical depth will be $\frac{2}{3}y_a$. The `Spillway` class has a method called `find_critical_discharge()` which computes the equation above to find $Q$, given $y$. Then we will use `find_normal_flow_depth()` to find out what the normal flow depth is at this discharge value. 

In [ ]:
yc = spillway.ya * 2/3

q = spillway.find_critical_discharge(yc)    # ~3.2 L/s 
b = spillway.b   # width of spillway
y = np.arange(0.046, 0.2, 0.001) # plot a range of depth values


fig, ax = plot_specific_energy_curve(y, q, spillway.b)

ax.plot(spec_E(yc, q, b), yc, 'o', color = 'xkcd:red')

ax.annotate(
    "Critical flow depth", 
    (spec_E(yc, q, b) * 1.1, yc), 
    rotation = 0, color = 'xkcd:red', ha = 'left', va = 'center'
)

yn = spillway.find_normal_flow_depth(q)

ax.plot(spec_E(yn, q, b), yn, 'o', color = 'xkcd:blue')

_ = ax.annotate(
    "Normal flow depth", 
    (spec_E(yn, q, b) * 1.2, yn), 
    rotation = 0, color = 'xkcd:blue', ha = 'left', va = 'center'
)

In this case, we can clearly see that the normal flow depth is less than the critical flow depth, so the specific energy of the flow will be greater, and the flow will be supercritical. This check confirms that the flow will be supercritical through the spillway, until it gets to the downstream end. If the level of the downstream pool is lower than the tailwater, then the jet will just enter the basin, but if it is higher, then we will expect a hydraulic jump.

By confirming that the flow will be supercritical, we can now use a few simplifying assumptions to compute the discharge. Supercritical flow implies that the downstream tail condition does not influence the flow at the upstream end at all (this is a good assumption as long as the downstream pool level is below the crest of the spillway. This scenario will be developed below). Thus, the discharge will just be the critical discharge, because the flow must pass through the critical condition on its way to establishing (supercritical) uniform flow. So, in this case, the value $Q_c$ we calculate from `find_critical_discharge()` is the correct discharge.

What about if the downstream pool level rises high enough to interfere with the water flow entering the spillway? This scenario is broadly speaking referred to as a "submerged weir," and there exists a variety of literature that allows one to correct for the submergence. In general, it becomes necessary if the downstream pool level is higher than the crest of the spillway, for example:

In [ ]:
spillway.set_y_values(0.15, spillway.za + 0.13) # set water levels
fig = spillway.show_current_condition()

In this situation, a simple and empirical way to do the correction is to apply a factor to the free-fall discharge (what we calculated before for the spillway) based on experimental data. A relevant USDA publication (USDA NEH-11 Design Note 15, May 1973, DWG NO. ES-207 Sheet 1) provides a correction factor $R$ that scales with $\frac{y_b - z_a}{y_a}$ which I will call $q_\star$ to distinguish it from the hydraulic radius. The formula is

$$ Q_s = q_\star Q_f$$

where $Q_s$ is the submerged discharge and $Q_f$ is the free-fall discharge. 

$$ q_\star = f\left (\frac{y_b - z_a}{y_a} \right ), $$

and that function is given graphically in this ES-207 Sheet 1, copied here below. 

<img src="../images/data/ES-207.png" alt="ES-207" style="width: 500px;"/>

This empirical function has been digitized and included with `Spillway` as a method (`submerge_interp()`) that provides interpolated values along this curve provided the submergence ratio $\frac{y_b - z_a}{y_a}$, and with the known geometry of the spillway system, this provides an appropriate correction to the free-fall discharge value. Thus, with the spillway configured as above, the following code shows the difference between the submerged and free-fall discharge values. 

In [ ]:
b = spillway.b   # width of spillway

yc = spillway.ya * 2/3

q = spillway.find_critical_discharge(yc)    # ~3.2 L/s 
q_star = spillway.submerge_interp((spillway.yb - spillway.za) / spillway.ya)
qs = q * q_star

print('Submergence ratio is: ' + str(round((spillway.yb - spillway.za) / spillway.ya, 4)))
print('q_star factor is: ' + str(round(float(q_star), 4)))

print('The predicted discharge is ' + str(round(1000 * q, 4)) + ' L/s')
print('The corrected discharge is ' + str(round(1000 * qs, 4)) + ' L/s')

It can be clearly seen that in this scenario, the weir must be substantially submerged before it begins to impact the flow of water over the weir. 

With these two components, we can now make a prediction for the discharge of the spillway whenever supercritical flow is involved. In practice, this means whenever the channel is very steep or whenever the flow becomes very thin at the inlet. Each of these scenarios is often encountered, so we now have a tool to predict the discharge in those scenarios. However, there are two other scenarios that merit our attention.

### The case of subcritical flow

If the spillway channel is shallow, then it is likely that normal flow will be established at a deep alternate depth in the critical flow scenario described above. For example, let's imagine the same scenario, but with a much shallower initial slope, and for convenience, a downstream pool that has water in it.

In [ ]:
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(4*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.11, 0.12) # set water levels
fig = spillway.show_current_condition() # method that displays the system state as a cartoon

If we assume that the critical depth is reached, and therefore the critical discharge assuming a $\frac{2}{3}$ criterion, we will predict a normal depth that is _higher_ than the critical depth.

In [ ]:
yc = spillway.ya * 2/3

q = spillway.find_critical_discharge(yc)    # ~2 L/s 
b = spillway.b   # width of spillway
y = np.arange(0.035, 0.18, 0.001) # plot a range of depth values

fig, ax = plot_specific_energy_curve(y, q, spillway.b)

ax.plot(spec_E(yc, q, b), yc, 'o', color = 'xkcd:red')

ax.annotate(
    "Critical flow depth", 
    (spec_E(yc, q, b) * 1.1, yc), 
    rotation = 0, color = 'xkcd:red', ha = 'left', va = 'center'
)

yn = spillway.find_normal_flow_depth(q)

ax.plot(spec_E(yn, q, b), yn, 'o', color = 'xkcd:blue')

_ = ax.annotate(
    "Normal flow depth", 
    (spec_E(yn, q, b) * 1.15, yn), 
    rotation = 0, color = 'xkcd:blue', ha = 'left', va = 'center'
)

This implies that the water has to thin to a critical depth and then thicken again to the normal depth downstream. This is not what happens in reality. In reality, the water surface rapidly approaches the normal depth at the upstream inlet, and eventually meets the downstream pool surface. This can happen either abruptly, if the downstream pool level is below the normal flow depth at the outlet of the channel, or asymptotically, if the downstream pool level is higher. 

Unfortunately, we do not know the discharge _a priori_, so we cannot know the normal depth either. If we assume that flow is steady, though, and the levels of the pools are constant through time, the water surface has to smoothly connect the two. If the water was the same depth throughout the spillway, the system would be in a state of normal flow, so $y_a = y_b$, and we could use the normal flow equation to calculate the discharge. If, however, $y_a \ne y_b$, then the flow will gradually vary along the spillway. If we make the assumption that the flow is gradually varying, but time-invariant, then a stable flow profile will develop between the two reservoirs according to the gradually-varied flow equation: 

$$ \frac{dy}{dx} = \frac{S_0 - S_f}{1 - Fr^2} $$

This equation does not have a general analytical solution, and is thus usually solved numerically. The equation can be written a number of different ways and parameterized with different assumptions. The version I use in this model can be found in Ven Te Chow, Chapter 9. So, to compute the flow profile, we solve this equation 

$$ \frac{dy}{dx} = S_0 \frac{1 - \frac{y_n}{y}^{\frac{10}{3}}\frac{2y + b}{2y_n + b}^\frac{4}{3}}{1 - \frac{y_c}{y}^3}$$

in the upstream direction. However, one will note that $y_n$ and $y_c$ are required parameters, so we must assume a discharge to compute the flow profile. Let's look at an example of what the flow profile looks like, so that we can see how we can make this work. So we will initialize a shallow sloping spillway system, set the pool levels, then assume a discharge, fix the downstream elevation to the downstream pool level, then compute the profile upstream. This gives us a value for the upstream depth of the flow profile. 

In [ ]:
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(4*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.11, 0.12) # set water levels

qlow = spillway.find_normal_discharge(spillway.ya) * 0.75   # ~2.4 L/s 

# this function solves the equation from the downstream end
x, y = spillway.compute_subcritical_flow_profile(qlow, spillway.yb) 

spillway.plot_flow_profile(qlow, x, y)

print('assumed low discharge is: ' + str(round(1000 * qlow, 3)) + ' L/s')
print('upstream depth is: ' + str(round(y[0], 4)))
print('downstream depth is: ' + str(round(y[-1], 4)))


Here, the black dashed line is the channel bed, the red dashed line is the critical depth, the black dashed line is the normal depth, and the blue line is the water surface, with dots showing the location of the upstream and downstream ends. In this case, as the data output show, the computed value for the upstream depth is too low. the value should be 0.11 m, but it is 0.0847 m. We can compute the upstream value for a number of different discharge conditions, and eventually find the one that matches the upstream depth. To do this, we once again use a root-finding algorithm, searching for a value of Q that simultaneously matches both $y_a$ and $y_b$. This function is `find_subcritical_discharge()` which takes the current status of the system, and searches between two different discharge values. 

It is important to pick the discharge bracket values to ensure that you actually bracket a real root. I have devised a few ways of making sure we find a real root when searching for this discharge value, some of these hueristics are not perfect, and will be worthwhile revisiting. See `compute_discharge_arbitrary()` for the control flow here. 


### Computing discharge with arbitrary head gradient

Nonetheless, we have arrived at an important milestone: being able to compute the discharge along the spillway given the channel geometry and the elevation of the two pools. There is a method to compute this: `compute_discharge_arbitrary()`, and it simply acts on the system object, returning the discharge and updating an object attribute `spillway.Q`. Take a look:

In [ ]:
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(4*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.11, 0.12) # set water levels
q_actual = spillway.compute_discharge_arbitrary()

print('computed discharge is: ' + str(round(1000 * q_actual, 3)) + ' L/s')

### Time-varying reservoirs

Now we have tackled the first big challenge: to be able to predict the discharge through the spillway given the water level of the two connected basins. Now, let's tackle the second of our main bullet points in the Modeling Strategy section. The challenge is now to build a time-evolving system, where the level of each reservoir changes as a result of the water flowing between them.

Let's call each reservoir $a$ and $b$, same as the upstream and downstream sections of the spillway. The rate of volume change for each reservoir is the discharge. 

$$ Q = -\frac{dV_a}{dt} = \frac{dV_b}{dt} $$

recalling that by convention, a flow out of the upper reservoir will make a positive discharge. Since each reservoir is roughly speaking a rectangle (excepting the small sliver of the spillway itself, but let's neglect that), the volume as a function of the depth in the reservoir is

$$ V_x = BL_xh_x $$

where $B$ is the flume width, $L$ is the length of that reservoir, and $h$ is the depth. $B$ and $L$ are both constants, so we can write

$$ Q = BL_x\frac{dh_x}{dt} $$

and since we are only measuring the rate, and since the upstream reservoir cannot drain below the crest of the spillway, we can re-cast this in terms of y

$$ Q = BL_x\frac{dy_x}{dt}. $$

So now we have a coupled system of ODEs where discharge is a complicated function of $y_a$ and $y_b$, and then the discharge gives the rate of change in those quantities.  

$$ \frac{dy_a}{dt} = -\frac{Q}{BL_a}, $$
$$ \frac{dy_b}{dt} = \frac{Q}{BL_b}, $$
$$ Q = f(y_a, y_b, \textrm{channel geometry}). $$

The system is written into a method called `spillway_ode()` and it the system is evolved from its current state with `evolve_system()`. 

The negative sign imparts a strong negative feedback here, so the system should easily evolve to a steady state. First, let's check out the simplest case, where no downstream effects can possibly impact the flow of the water, because there is not enough water to possibly fill the downstream reservoir, and the flow is certainly supercritical the entire time. 

In [ ]:
%autoreload
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(8*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.11, 0.0) # set water levels
spillway.show_current_condition()

In [ ]:
t, y = spillway.evolve_system(t = 300)

In [ ]:
plt.plot(t, y.transpose() + [spillway.zb, spillway.za], '.-')
plt.xlabel('time (s)')
plt.ylabel('reservoir elevation (m)')
plt.legend(['downstream', 'upstream'])

In [ ]:
spillway.show_current_condition()

What is clear here is that enough water drained in about 2-3 minutes from the top to the bottom reservoir such that flow has almost completely stopped. This outcome is pretty straightforward, and the main thing one would have to calibrate in the lab is the roughness coefficient, which is essential in these computations. So what one would have to do is measure the time for the system to evolve to steady state, and measure the water level in each pool, and then find the roughness value that fits the data best. 

What about when the water level in the downstream basin can exceed the top of the spillway crest? Here is that scenario. It should be clear that there is more than enough water, so the water level should equilibrate somewhere between the two pool levels, but over the crest of the spillway. 

In [ ]:
spillway.set_y_values(0.13, 0.17) # set water levels
spillway.show_current_condition()

In [ ]:
t, y = spillway.evolve_system(t = 100)

plt.plot(t, y.transpose() + [spillway.zb, spillway.za], '.-')
plt.xlabel('time (s)')
plt.ylabel('reservoir elevation (m)')
plt.legend(['downstream', 'upstream'])
spillway.show_current_condition()

Et voilà! The behavior here is also reasonable, but not verified against experiments. 

The next scenario we should check is the subcritical flow case, where the physics are similar, but not always as straighforward, because sometimes the flow becomes supercritical when you were not expecting it to _a priori_.

Considering the same setup, what does the behavior look like if we take a shallow-sloping spillway and start it out dry at the downstream end?

In [ ]:
%autoreload
spillway = sp.Spillway() # initialize the spillway object
spillway.set_defaults() # set a range of reasonable defaults.
spillway.set_za(4*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.08, 0.0) # set water levels
spillway.show_current_condition()

In [ ]:
t, y = spillway.evolve_system(t = 300)

plt.plot(t, y.transpose() + [spillway.zb, spillway.za], '.-')
plt.xlabel('time (s)')
plt.ylabel('reservoir elevation (m)')
plt.legend(['downstream', 'upstream'])
spillway.show_current_condition()

The behavior is much the same if the slope is shallow, but there is water in each pool.

In [ ]:
spillway.set_za(4*2.54*1e-2) # set up the spillway with elevation fall of 4 inches
spillway.set_y_values(0.1, 0.08) # set water levels
spillway.show_current_condition()

In [ ]:
t, y = spillway.evolve_system(t = 100, time_step=10)

plt.plot(t, y.transpose() + [spillway.zb, spillway.za], '.-')
plt.xlabel('time (s)')
plt.ylabel('reservoir elevation (m)')
plt.legend(['downstream', 'upstream'])
spillway.show_current_condition()

## Summary and utility for future work

So to sum up, we now have a system that can evolve forward in time, and does so while considering the elevation of two pools that co-evolve while discharge moves between them. In the future, it should be relatively simple to predict the discharge regime of a system where the downstream lake level is pinned. 

This model will be useful for constraining the roughness (Manning's n) of the system, as well as checking some of the mechanisms for predicting flow, and maybe selecting better equations, even if they are empirical. 

When we do incorporate an erodible bed into the experiments, this system will allow us to make some guesses about how much the erosion/deposition is affecting the hydraulics of the spillway and the flood wave itself. This model is essentially a null hypothesis to compare agains when working in the flume. 